In [61]:
sys.path.append('..')
from config import Config

import json
import numpy as np # Для работы с данными 
import sys
import os

from tensorflow.keras import utils # Для работы с категориальными данными
from tensorflow.keras.preprocessing.text import Tokenizer # Методы для работы с текстами

In [62]:
JSON_DATA_DIR = '../../Satellites'

In [88]:
# создаём два списка, в пустые элементы которых будем добавлять тексты
# в первом элементе будет храниться текст с ответом False, во втором - True
textClasses = ['', '']
testTextClasses = []

# открываем файл с id эссе и ответами
with open(JSON_DATA_DIR + '/train/train_standart.json', 'r') as f_list:
  data = json.load(f_list)

  # проходимся по каждому "блоку" с эссе
  for i in range(len(data)):
    elem = data[i]
    with open(JSON_DATA_DIR + f'/train/essays/{elem["id"]}.json', 'r') as essay:
      file = json.load(essay)
      text = file['text']
      if i < 114:
        testTextClasses.append([[text], elem["answer"]])
        continue
      if elem['answer'] == False:
        textClasses[0] += text
        textClasses[0] += '#'
      else:
        textClasses[1] += text
        textClasses[1] += '#'

print(testTextClasses[0])

[['Считаю, почти все дадут согласие, что в протяжении жизни мы ни один раз встречаем безответственных профессионалов собственного дела. К несчастью, если дело касается медицины, то безалаберное отношение может сыграть злую шуточку. Над неувязкой ответственности анализирует автор текста. Василий Павлович Аксенов думает, что к собственной работе нужно относится серьезно и радиво, в особенности если идет речь о здоровье остальных людей. Свое мнение автор обосновывает на примере ситуации из текста.\nВасилий Павлович Аксенов думает, что к собственной работе нужно относится серьезно и радиво, в особенности если идет речь о здоровье остальных людей. Свое мнение автор обосновывает на примере ситуации из текста. Василий Павлович противопоставляет 2-ух мед сотрудников различных поколений. Юный спец Александр Зеленин чрезвычайно трепетно подступает к реализации собственных обязательств. Он не может «закрыть глаза» на почти все упущения. Автор подчеркивает это в предложениях 2,3. Не считая того св

In [106]:
def getSetFromIndexes(wordIndexes, xLen, step):
  xText = []
  wordsLen = len(wordIndexes) # Считаем количество слов
  index = 0 # Задаем начальный индекс 

  while (index + xLen <= wordsLen): # Идём по всей длине вектора индексов
    xText.append(wordIndexes[index:index+xLen]) # "Откусываем" векторы длины xLen
    index += step # Смещаемся вперёд на step
    
  return xText

def createSetsMultiClasses(wordIndexes, xLen, step): # Функция принимает последовательность индексов, размер окна, шаг окна
  nClasses = len(wordIndexes) # Количество классов
  classesXSamples = []        # Здесь будет список размером "кол-во классов*кол-во окон в тексте*длину окна"
  for wIclass in range(nClasses):      # Для каждого текста выборки из последовательности индексов
    if wIclass == 0:
      classesXSamples.append(getSetFromIndexes(wordIndexes[0], xLen, step[0]))
    else:
      classesXSamples.append(getSetFromIndexes(wordIndexes[1], xLen, step[1]))

  # Формируем один общий xSamples
  xSamples = []
  ySamples = []
  
  for t in range(nClasses):
    xT = classesXSamples[t]
    for i in range(len(xT)): # Перебираем каждое окно определенного класса
      xSamples.append(xT[i]) # Добавляем в общий список выборки
      ySamples.append(utils.to_categorical(t, nClasses)) # Добавляем соответствующий вектор класса

  xSamples = np.array(xSamples)
  ySamples = np.array(ySamples)

  print(xSamples.shape)

  return (xSamples, ySamples)

In [119]:
def getSetFromIndexes(wordIndexes, xLen, step):
  xText = []
  wordsLen = len(wordIndexes) # Считаем количество слов
  index = 0 # Задаем начальный индекс 

  while (index + xLen <= wordsLen): # Идём по всей длине вектора индексов
    xText.append(wordIndexes[index:index+xLen]) # "Откусываем" векторы длины xLen
    index += step # Смещаемся вперёд на step
    
  return xText

def createSetsMultiClassesTest(wordIndexes, xLen, step): # Функция принимает последовательность индексов, размер окна, шаг окна
  nClasses = len(wordIndexes) # Количество классов
  classesXSamples = []        # Здесь будет список размером "кол-во классов*кол-во окон в тексте*длину окна"
  for wI in wordIndexes:      # Для каждого текста выборки из последовательности индексов
    classesXSamples.append(getSetFromIndexes(wordIndexes[0], xLen, step))


  # Формируем один общий xSamples
  xSamples = []
  ySamples = []
  
  for t in range(nClasses):
    xT = classesXSamples[t]
    for i in range(len(xT)): # Перебираем каждое окно определенного класса
      xSamples.append(xT[i]) # Добавляем в общий список выборки
      ySamples.append(utils.to_categorical(t, nClasses)) # Добавляем соответствующий вектор класса

  xSamples = np.array(xSamples)
  ySamples = np.array(ySamples)

  return (xSamples, ySamples)

In [65]:
labels = ['False', 'True']
labelsNum = len(labels)

In [92]:
texts_false = textClasses[0].split("#")
texts_true = textClasses[1].split("#")

test_border_false = len(texts_false)//10
test_border_true = len(texts_true)//10

valText = []
valText.append(' '.join(texts_false[:test_border_false]))
valText.append(' '.join(texts_true[:test_border_true]))


trainText = []
trainText.append(' '.join(texts_false))
trainText.append(' '.join(texts_true))

In [116]:
tokenizer = Tokenizer(
    filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff',
    lower=True,
    split=' ',
    oov_token='unknown',
    char_level=False)

tokenizer.fit_on_texts(trainText)

print(len(list(tokenizer.word_index.items())))

49208


In [122]:
numWords = 49208
xLen = 300
step = (150, 35)

trainWordIndexes = tokenizer.texts_to_sequences(trainText)
valWordIndexes = tokenizer.texts_to_sequences(valText)


xTrainId, yTrain = createSetsMultiClasses(trainWordIndexes, xLen, step)
xValId, yVal = createSetsMultiClasses(valWordIndexes, xLen, step)

for i in range(len(testTextClasses)):
    testTextArray = tokenizer.texts_to_sequences(testTextClasses[i][0])
    xTestId, yTest = createSetsMultiClassesTest(testTextArray, xLen, xLen)
    xTest = xTestId
    break


(3737, 300)
(402, 300)


In [123]:

np.savez_compressed(f"{Config.DATASETS_DIR}NEWNEWdataset_numWords_{numWords}_xLen_{xLen}_step_{xLen}_.npz", xTrain=xTrainId, yTrain=yTrain, xVal=xValId, yVal=yVal, xTest=xTest, yTest=yTest)
